# 内存管理

内存管理由 Python 解释器自动处理，开发者无需显式地进行内存分配和释放。内存管理机制主要包括：


## 对象缓存优化

针对特定不可变类型对象在内存中缓存、重用。避免频繁创建/销毁对象，减少内存分配开销。


**小整数缓存**

`[-5, 256]` 区间的小整数会被缓存。


In [281]:
x = 256
y = 256
x is y

True

**字符串驻留**

由字母、数字、下划线组成的字符串会被缓存。


In [282]:
a = "hello_world_001"
b = "hello_world_001"
a is b

True

> `sys.intern` 方法可手动创建驻留字符串。


In [283]:
import sys

a = sys.intern("你好")
b = sys.intern("你好")
a is b

True

**空元组缓存**


In [284]:
a = ()
b = ()
a is b

True

## 垃圾回收


**引用计数**

每当对象被引用时，引用计数加 1；当引用被删除或超出作用域时，引用计数减 1。
当引用计数变为 0 时，对象将被立即回收。

> 两个或多个对象之间相互引用形成一个环即为循环引用，引用计数机制无法处理对象循环引用。

> `sys.getrefcount` 方法可获取对象引用计数。注意 `sys.getrefcount` 方法本身会增加一个对象临时引用。


In [285]:
from random import random
import sys

x = random()  # 生成一个没有被内存缓存的随机数

sys.getrefcount(x)  # 获取对象引用计数

2

In [286]:
sys.getsizeof(x)  # 获取对象占用内存字节数

24

In [287]:
ref = x

sys.getrefcount(x)

3

In [288]:
del ref  # 删除一次引用

sys.getrefcount(x)

2